# Ensemble prediction

    ### Train `num` independt models, then sum the prediction probability, then `argmax` it to the prediction result
## 1. prepare the model
## 2. train our ensemble model
    ### Training result 1
    ### Training result 2

## 4. Compare the result of trainging with dropout between the conv. layers
    ### Training result 
    ### Training result (without)   

## 3. without dropout between the convolution layer
    ### Training result 1
    ### Training result 2
    
    
What's more, we find that training the model with **pycharm** instead the jupyter notebook, we can get faster training

## 1. prepare the model

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# hyper parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100


class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])

            # img 28x28x1 (black/white), Input Layer
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,
                                         rate=0.7, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,
                                         rate=0.7, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3],
                                     padding="same", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                                            padding="same", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,
                                         rate=0.7, training=self.training)

            # Dense Layer with Relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs=flat,
                                     units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,
                                         rate=0.5, training=self.training)

            # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout4, units=10)

        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(
            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X: x_test,
                                        self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})

# initialize
sess = tf.Session()

models = []
num_models = 7
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning Started!


## 2. train our ensemble model

In [ ]:

for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')

# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros(test_size * 10).reshape(test_size, 10)
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

### Training result 1

In [ ]:
Learning Started!
Epoch: 0001 cost = [ 0.8285886   0.7736028   0.7857488   0.75940615  0.79814789  0.78667722
  0.80067089]
Epoch: 0002 cost = [ 0.29876267  0.29312797  0.28643643  0.29918091  0.29994753  0.28521119
  0.29203279]
Epoch: 0003 cost = [ 0.23014378  0.23366693  0.22784479  0.24353373  0.2370689   0.22865331
  0.22992973]
Epoch: 0004 cost = [ 0.20278791  0.20509444  0.19784418  0.2094885   0.20768992  0.20207432
  0.20021661]
Epoch: 0005 cost = [ 0.18334771  0.18573112  0.18400545  0.19082383  0.18628724  0.18291346
  0.18421995]
Epoch: 0006 cost = [ 0.17065732  0.17457147  0.17230677  0.17679305  0.17644136  0.17437257
  0.17084923]
Epoch: 0007 cost = [ 0.16017703  0.16466835  0.16185024  0.1692534   0.16537071  0.16400755
  0.15925765]
Epoch: 0008 cost = [ 0.16058975  0.16071422  0.15743243  0.15889423  0.15913958  0.15940014
  0.1582038 ]
Epoch: 0009 cost = [ 0.1507076   0.15051623  0.15180255  0.1544444   0.15662055  0.1518818
  0.15450658]
Epoch: 0010 cost = [ 0.14310655  0.15382333  0.14273362  0.14761922  0.15230439  0.1551467
  0.14623267]
Epoch: 0011 cost = [ 0.14428991  0.14660035  0.14505644  0.14935583  0.1460829   0.14696486
  0.14405732]
Epoch: 0012 cost = [ 0.14016805  0.14399414  0.14186701  0.14143713  0.13962515  0.14726369
  0.14257362]
Epoch: 0013 cost = [ 0.14083112  0.13810326  0.13711051  0.14152663  0.14216629  0.13932477
  0.13818365]
Epoch: 0014 cost = [ 0.13783987  0.13700243  0.13507878  0.13790451  0.13980047  0.13975731
  0.13374982]
Epoch: 0015 cost = [ 0.13302495  0.13308863  0.13490895  0.13805266  0.13715017  0.13733935
  0.13263847]
Epoch: 0016 cost = [ 0.13662738  0.13232288  0.13476929  0.13810129  0.13736914  0.13784942
  0.1357136 ]
Epoch: 0017 cost = [ 0.13206882  0.13110494  0.13224021  0.13533757  0.13055664  0.13765583
  0.13062567]
Epoch: 0018 cost = [ 0.1296537   0.13087357  0.13160314  0.13508857  0.13237516  0.13556652
  0.13377178]
Epoch: 0019 cost = [ 0.13263026  0.12800627  0.12662523  0.12929243  0.13390452  0.131273
  0.131616  ]
Epoch: 0020 cost = [ 0.12875591  0.12947235  0.12952766  0.13245394  0.13097352  0.13090216
  0.12515859]
Learning Finished!
0 Accuracy: 0.9896
1 Accuracy: 0.9906
2 Accuracy: 0.9904
3 Accuracy: 0.9903
4 Accuracy: 0.9895
5 Accuracy: 0.9894
6 Accuracy: 0.9894
Ensemble accuracy: 0.9908

Process finished with exit code 0

### Training result 2

In [ ]:
/home/cine/anaconda3/bin/python /home/cine/PycharmProjects/CNN_MNIST/ensemble.py
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcurand.so.8.0 locally
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:910] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
I tensorflow/core/common_runtime/gpu/gpu_device.cc:885] Found device 0 with properties: 
name: GeForce GTX 1080
major: 6 minor: 1 memoryClockRate (GHz) 1.898
pciBusID 0000:01:00.0
Total memory: 7.91GiB
Free memory: 7.08GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:906] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:916] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:975] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0)
Learning Started!
Epoch: 0001 cost = [ 0.83047655  0.77355436  0.78631724  0.76047829  0.79808162  0.78900232
  0.80444025]
Epoch: 0002 cost = [ 0.29044883  0.29349009  0.28415209  0.30511432  0.29380412  0.291963
  0.30093969]
Epoch: 0003 cost = [ 0.22927809  0.2333655   0.22727925  0.24126358  0.23425651  0.22951297
  0.23469908]
Epoch: 0004 cost = [ 0.19967717  0.19799492  0.19886177  0.21057255  0.20531667  0.19882518
  0.20526784]
Epoch: 0005 cost = [ 0.18194842  0.18384478  0.18303141  0.18926625  0.18769689  0.18053432
  0.18367032]
Epoch: 0006 cost = [ 0.1697628   0.17170594  0.16899526  0.17634839  0.17712699  0.17743356
  0.17552089]
Epoch: 0007 cost = [ 0.15931895  0.16245569  0.16304636  0.17365431  0.16218049  0.16408003
  0.16307671]
Epoch: 0008 cost = [ 0.15601101  0.15762019  0.16068577  0.15967464  0.16041245  0.1603231
  0.15589744]
Epoch: 0009 cost = [ 0.15027691  0.1523215   0.15514017  0.15566602  0.15634622  0.1526949
  0.15189681]
Epoch: 0010 cost = [ 0.15180895  0.14623143  0.1476934   0.14954521  0.15292802  0.14985852
  0.14636758]
Epoch: 0011 cost = [ 0.14461594  0.14300818  0.14276493  0.14791104  0.14803213  0.14838753
  0.14392576]
Epoch: 0012 cost = [ 0.14331938  0.14650698  0.14364925  0.14739382  0.14515571  0.1436125
  0.14076496]
Epoch: 0013 cost = [ 0.138173    0.14032773  0.136132    0.13803601  0.14494504  0.13717389
  0.1355785 ]
Epoch: 0014 cost = [ 0.13638169  0.13852525  0.13734646  0.13546502  0.13478948  0.13970664
  0.13757044]
Epoch: 0015 cost = [ 0.13387798  0.13417005  0.137983    0.1342857   0.1365686   0.13471815
  0.1334124 ]
Epoch: 0016 cost = [ 0.13254187  0.13100623  0.13410034  0.13301066  0.13512878  0.13753826
  0.13152365]
Epoch: 0017 cost = [ 0.13427727  0.13027125  0.13186019  0.132072    0.13242968  0.13134369
  0.13102028]
Epoch: 0018 cost = [ 0.13095213  0.13152373  0.12951459  0.13151891  0.12841374  0.13113056
  0.12526906]
Epoch: 0019 cost = [ 0.12722736  0.1289917   0.13149611  0.12912504  0.12866268  0.13212123
  0.13269841]
Epoch: 0020 cost = [ 0.12603178  0.13017603  0.12472516  0.13023419  0.13013125  0.13106258
  0.12984498]
Learning Finished!
0 Accuracy: 0.9902
1 Accuracy: 0.9908
2 Accuracy: 0.9895
3 Accuracy: 0.9907
4 Accuracy: 0.991
5 Accuracy: 0.9899
6 Accuracy: 0.9896
Ensemble accuracy: 0.9918

Process finished with exit code 0


# Compare the result of trainging with dropout between the conv. layers
### Training result 

In [ ]:
0 Accuracy: 0.9902
1 Accuracy: 0.9908
2 Accuracy: 0.9895
3 Accuracy: 0.9907
4 Accuracy: 0.991
5 Accuracy: 0.9899
6 Accuracy: 0.9896
Ensemble accuracy: 0.9918
    
Learning Finished!
0 Accuracy: 0.9896
1 Accuracy: 0.9906
2 Accuracy: 0.9904
3 Accuracy: 0.9903
4 Accuracy: 0.9895
5 Accuracy: 0.9894
6 Accuracy: 0.9894
Ensemble accuracy: 0.9908

### Training result (without)   

In [ ]:
Learning Finished!
0 Accuracy: 0.9948
1 Accuracy: 0.994
2 Accuracy: 0.9928
3 Accuracy: 0.994
4 Accuracy: 0.9947
5 Accuracy: 0.995
6 Accuracy: 0.9941
Ensemble accuracy: 0.9953
    
Learning Finished!
0 Accuracy: 0.9952
1 Accuracy: 0.9944
2 Accuracy: 0.9937
3 Accuracy: 0.9942
4 Accuracy: 0.9939
5 Accuracy: 0.9955
6 Accuracy: 0.9945
Ensemble accuracy: 0.9956

# 3. Without dropout between the convolution layer

In [ ]:

import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# hyper parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100


class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])

            # img 28x28x1 (black/white), Input Layer
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            # dropout1 = tf.layers.dropout(inputs=pool1,
            #                              rate=0.7, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            # dropout2 = tf.layers.dropout(inputs=pool2,
            #                              rate=0.7, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=[3, 3],
                                     padding="same", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                                            padding="same", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,
                                         rate=0.7, training=self.training)

            # Dense Layer with Relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs=flat,
                                     units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,
                                         rate=0.5, training=self.training)

            # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout4, units=10)

        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(
            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X: x_test,
                                        self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})


# initialize
sess = tf.Session()

models = []
num_models = 7
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # train each model
        for m_idx, m in enumerate(models):# -*- coding: utf-8 -*-
# __author__ = "CHG"
# __createTime__ = '5/5/17'
# Email: kwchenghong@gmail.com
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# hyper parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100


class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])

            # img 28x28x1 (black/white), Input Layer
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            # dropout1 = tf.layers.dropout(inputs=pool1,
            #                              rate=0.7, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            # dropout2 = tf.layers.dropout(inputs=pool2,
            #                              rate=0.7, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=[3, 3],
                                     padding="same", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                                            padding="same", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,
                                         rate=0.7, training=self.training)

            # Dense Layer with Relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs=flat,
                                     units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,
                                         rate=0.5, training=self.training)

            # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout4, units=10)

        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(
            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X: x_test,
                                        self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})


# initialize
sess = tf.Session()

models = []
num_models = 7
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')

# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros(test_size * 10).reshape(test_size, 10)
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')

# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros(test_size * 10).reshape(test_size, 10)
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))


### Training result 1(without)

In [ ]:
/home/cine/anaconda3/bin/python /home/cine/PycharmProjects/CNN_MNIST/ensemble2.py
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcurand.so.8.0 locally
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:910] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
I tensorflow/core/common_runtime/gpu/gpu_device.cc:885] Found device 0 with properties: 
name: GeForce GTX 1080
major: 6 minor: 1 memoryClockRate (GHz) 1.898
pciBusID 0000:01:00.0
Total memory: 7.91GiB
Free memory: 7.09GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:906] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:916] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:975] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0)
Learning Started!
Epoch: 0001 cost = [ 0.27680884  0.28473268  0.29844935  0.28272435  0.28920459  0.29054749
  0.2887564 ]
Epoch: 0002 cost = [ 0.09735898  0.09161068  0.09377506  0.09207193  0.09295438  0.09206543
  0.09443576]
Epoch: 0003 cost = [ 0.07206015  0.06818733  0.07169108  0.06935435  0.06900203  0.07044479
  0.07124836]
Epoch: 0004 cost = [ 0.06149847  0.05680746  0.06076545  0.06161918  0.05839696  0.0576337
  0.0590065 ]
Epoch: 0005 cost = [ 0.05381385  0.05125328  0.05252606  0.05274087  0.05048139  0.05205292
  0.0522524 ]
Epoch: 0006 cost = [ 0.0473631   0.04484748  0.04715837  0.04439492  0.04574615  0.04498552
  0.04781991]
Epoch: 0007 cost = [ 0.0446623   0.04227922  0.04209529  0.04279164  0.04226583  0.04147751
  0.04379641]
Epoch: 0008 cost = [ 0.0420294   0.03818532  0.03937593  0.0382924   0.03890409  0.03651301
  0.03834822]
Epoch: 0009 cost = [ 0.03697141  0.03676942  0.03796514  0.03641365  0.03713465  0.03345095
  0.03604359]
Epoch: 0010 cost = [ 0.03488226  0.03521734  0.03289332  0.03632225  0.0323164   0.03282443
  0.03401624]
Epoch: 0011 cost = [ 0.03125854  0.03214217  0.03146074  0.03352601  0.03328456  0.03227083
  0.03287639]
Epoch: 0012 cost = [ 0.02968449  0.03026564  0.0322317   0.02960546  0.02803696  0.02834099
  0.03111151]
Epoch: 0013 cost = [ 0.02974757  0.02791822  0.02769843  0.02910324  0.02782573  0.0281397
  0.02740344]
Epoch: 0014 cost = [ 0.02940068  0.02809145  0.02745237  0.02796813  0.02638819  0.02450916
  0.02707688]
Epoch: 0015 cost = [ 0.02639095  0.02558048  0.02610494  0.02528079  0.02496747  0.02611854
  0.02757929]
Epoch: 0016 cost = [ 0.02452684  0.02454993  0.02435415  0.02746548  0.02591348  0.02430854
  0.0252154 ]
Epoch: 0017 cost = [ 0.02636152  0.02259956  0.02403106  0.02418848  0.02346992  0.0233809
  0.02408849]
Epoch: 0018 cost = [ 0.02359937  0.0240713   0.0254393   0.02333885  0.02135288  0.02136151
  0.02308267]
Epoch: 0019 cost = [ 0.02268584  0.02340459  0.02195712  0.02281069  0.02105249  0.0233533
  0.02270263]
Epoch: 0020 cost = [ 0.02245908  0.02067757  0.02091201  0.02039999  0.0210384   0.02052974
  0.02299357]
Learning Finished!
0 Accuracy: 0.9952
1 Accuracy: 0.9944
2 Accuracy: 0.9937
3 Accuracy: 0.9942
4 Accuracy: 0.9939
5 Accuracy: 0.9955
6 Accuracy: 0.9945
Ensemble accuracy: 0.9956

Process finished with exit code 0


### Training result 2(without)

In [ ]:
/home/cine/anaconda3/bin/python /home/cine/PycharmProjects/CNN_MNIST/ensemble2.py
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcurand.so.8.0 locally
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:910] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
I tensorflow/core/common_runtime/gpu/gpu_device.cc:885] Found device 0 with properties: 
name: GeForce GTX 1080
major: 6 minor: 1 memoryClockRate (GHz) 1.898
pciBusID 0000:01:00.0
Total memory: 7.91GiB
Free memory: 6.92GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:906] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:916] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:975] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0)
Learning Started!
Epoch: 0001 cost = [ 0.27896288  0.28449352  0.29841569  0.2835605   0.28947163  0.28952805
  0.28778763]
Epoch: 0002 cost = [ 0.09733404  0.09115166  0.09258227  0.09402824  0.09172092  0.09266431
  0.09220581]
Epoch: 0003 cost = [ 0.07203782  0.07086627  0.0721186   0.0691958   0.07018349  0.06762542
  0.06911754]
Epoch: 0004 cost = [ 0.05874403  0.06109618  0.06024651  0.06041027  0.0583746   0.05944427
  0.06089767]
Epoch: 0005 cost = [ 0.05257655  0.0521298   0.05208349  0.05196292  0.04994644  0.04998246
  0.04959187]
Epoch: 0006 cost = [ 0.04893311  0.04803214  0.04703087  0.0469461   0.04695686  0.04476551
  0.0454702 ]
Epoch: 0007 cost = [ 0.04417387  0.04193157  0.04220696  0.041465    0.04144732  0.04134626
  0.0400673 ]
Epoch: 0008 cost = [ 0.03962938  0.03990052  0.03950938  0.04098858  0.03755157  0.03760113
  0.0402993 ]
Epoch: 0009 cost = [ 0.03705844  0.03853027  0.03734854  0.03718832  0.03607053  0.03682194
  0.03581756]
Epoch: 0010 cost = [ 0.03406331  0.03181378  0.03280855  0.03528593  0.03234821  0.03406168
  0.03532582]
Epoch: 0011 cost = [ 0.03138393  0.03134186  0.03308107  0.03271678  0.03154335  0.03052479
  0.0327291 ]
Epoch: 0012 cost = [ 0.03241862  0.028882    0.02906032  0.03033578  0.02943393  0.02843441
  0.0293292 ]
Epoch: 0013 cost = [ 0.03093915  0.02937761  0.0286764   0.02788935  0.02761049  0.02920519
  0.02787066]
Epoch: 0014 cost = [ 0.02992337  0.02730394  0.02601471  0.02775679  0.02754987  0.02814648
  0.02782688]
Epoch: 0015 cost = [ 0.02583008  0.02634076  0.02666942  0.02722986  0.02790946  0.02357761
  0.02762701]
Epoch: 0016 cost = [ 0.0240907   0.02504775  0.02584813  0.0260319   0.02144041  0.02669045
  0.02396525]
Epoch: 0017 cost = [ 0.02595522  0.02481141  0.02447678  0.02359887  0.02335667  0.02383656
  0.02263436]
Epoch: 0018 cost = [ 0.02362997  0.02075641  0.02376856  0.02258869  0.02292579  0.02329117
  0.02282054]
Epoch: 0019 cost = [ 0.02402067  0.02285908  0.02175709  0.022044    0.02130014  0.02163807
  0.02321238]
Epoch: 0020 cost = [ 0.02004196  0.02110491  0.02180612  0.0213354   0.0217759   0.02016811
  0.0228333 ]
Learning Finished!
0 Accuracy: 0.9948
1 Accuracy: 0.994
2 Accuracy: 0.9928
3 Accuracy: 0.994
4 Accuracy: 0.9947
5 Accuracy: 0.995
6 Accuracy: 0.9941
Ensemble accuracy: 0.9953

Process finished with exit code 0